In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))


In [3]:
df = pd.read_csv('titanic/test.csv')
df_row = df.copy()
df.head()
df['Family_Size'] = df['SibSp'] + df['Parch']
df.drop(['SibSp','Parch', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
df['Title'] = 0
for i in df:
    df['Title'] = df_row['Name'].str.extract('([A-Za-z]+)\.', expand = False) #uzywam Regex zeby wyciagnac tytul
df.head()
means = df.groupby('Title')['Age'].mean()
map_means = means.to_dict()
map_means
idx_null_age = df.loc[pd.isnull(df['Age'])].index
df.loc[idx_null_age,'Age'] = df['Title'].loc[idx_null_age].map(map_means)
df['Imputed'] = 0
df.at[idx_null_age.values, 'Imputed'] = 1

In [4]:
df['Title'] = pd.Categorical(df['Title'])
titles_dict = {'Capt': 'Other',
               'Major': 'Other',
               'Jonkheer': 'Other',
               'Don': 'Other',
               'Sir': 'Other',
               'Dr': 'Other',
               'Rev': 'Other',
               'Countess': 'Other',
               'Dona': 'Other',
               'Mme': 'Mrs',
               'Mlle': 'Miss',
               'Ms': 'Miss',
               'Mr': 'Mr',
               'Mrs': 'Mrs',
               'Miss': 'Miss',
               'Master': 'Master',
               'Lady': 'Other'}
df['Title'] = df['Title'].map(titles_dict)
df['Sex'] = pd.Categorical(df['Sex'])
df['Age'] = pd.cut(df['Age'], [0,15,50,200], labels = ['Child', 'Adult', 'Elder'])
df.drop('PassengerId', axis = 1, inplace = True)
df['Embarked'] = pd.Categorical(df['Embarked'])
df['Pclass'] = pd.Categorical(df['Pclass'])
df = pd.get_dummies(df, drop_first = 1)
df.dtypes
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
X_test = df.copy()
X_test.head()
X_test_transf = X_test.copy()
index = X_test_transf[pd.isnull(X_test_transf['Fare'])].index
X_test_transf['Fare'].fillna(X_test_transf['Fare'].mean(), inplace = True)
X_test_transf['Fare'][152]

35.6271884892086

In [5]:
X_test_transf['Fare'] = boxcox(X_test_transf['Fare'] + 1)[0]
scaler = MinMaxScaler()
X_test_transformed_scaled = scaler.fit_transform(X_test_transf)
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 2).fit(X_test_transf)
X_test_poly = poly.transform(X_test_transformed_scaled)
print(poly.get_feature_names())

['1', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x0^2', 'x0 x1', 'x0 x2', 'x0 x3', 'x0 x4', 'x0 x5', 'x0 x6', 'x0 x7', 'x0 x8', 'x0 x9', 'x0 x10', 'x0 x11', 'x0 x12', 'x0 x13', 'x1^2', 'x1 x2', 'x1 x3', 'x1 x4', 'x1 x5', 'x1 x6', 'x1 x7', 'x1 x8', 'x1 x9', 'x1 x10', 'x1 x11', 'x1 x12', 'x1 x13', 'x2^2', 'x2 x3', 'x2 x4', 'x2 x5', 'x2 x6', 'x2 x7', 'x2 x8', 'x2 x9', 'x2 x10', 'x2 x11', 'x2 x12', 'x2 x13', 'x3^2', 'x3 x4', 'x3 x5', 'x3 x6', 'x3 x7', 'x3 x8', 'x3 x9', 'x3 x10', 'x3 x11', 'x3 x12', 'x3 x13', 'x4^2', 'x4 x5', 'x4 x6', 'x4 x7', 'x4 x8', 'x4 x9', 'x4 x10', 'x4 x11', 'x4 x12', 'x4 x13', 'x5^2', 'x5 x6', 'x5 x7', 'x5 x8', 'x5 x9', 'x5 x10', 'x5 x11', 'x5 x12', 'x5 x13', 'x6^2', 'x6 x7', 'x6 x8', 'x6 x9', 'x6 x10', 'x6 x11', 'x6 x12', 'x6 x13', 'x7^2', 'x7 x8', 'x7 x9', 'x7 x10', 'x7 x11', 'x7 x12', 'x7 x13', 'x8^2', 'x8 x9', 'x8 x10', 'x8 x11', 'x8 x12', 'x8 x13', 'x9^2', 'x9 x10', 'x9 x11', 'x9 x12', 'x9 x13', 'x10^2', 'x10 x11', '

In [6]:
X_test_to_model = X_test_poly[:, 0:74]
results = loaded_model.predict(X_test_to_model)
predictions = pd.Series(results)
predictions.head()
real_survived = pd.read_csv('titanic/gender_submission.csv')
real_survived.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [7]:
comp_table = pd.concat([real_survived, predictions], axis = 1)
comp_table.rename(columns = {0:'Prediction'}, inplace = True)
comp_table['Good'] = 0
comp_table.at[comp_table[comp_table['Survived'] == comp_table['Prediction']].index, 'Good'] = 1
percent_score = comp_table['Good'].sum()/comp_table['PassengerId'].count()
print('percent score is: %.3f' % percent_score)

percent score is: 0.926


In [8]:
comp_table.head(30)

,PassengerId,Survived,Prediction,Good
0,892,0,0,1
1,893,1,1,1
2,894,0,0,1
3,895,0,0,1
4,896,1,0,0
5,897,0,0,1
6,898,1,1,1
7,899,0,0,1
8,900,1,1,1
9,901,0,0,1
